In [2]:
import cv2
import mediapipe as mp
import argparse

def process_img(img, face_detection):
    H, W, _ = img.shape
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    out = face_detection.process(img_rgb)

    if out.detections:
        for detection in out.detections:
            bbox = detection.location_data.relative_bounding_box

            x1 = int(bbox.xmin * W)
            y1 = int(bbox.ymin * H)
            w  = int(bbox.width * W)
            h  = int(bbox.height * H)

            # Apply blur safely inside image boundaries
            x1 = max(0, x1)
            y1 = max(0, y1)
            x2 = min(W, x1 + w)
            y2 = min(H, y1 + h)

            img[y1:y2, x1:x2] = cv2.blur(img[y1:y2, x1:x2], (50, 50))

    return img


# --------------------------
# Argument parser
# --------------------------
parser = argparse.ArgumentParser()
parser.add_argument("--mode", default="webcam")   # real-time default
parser.add_argument("--filePath", default="./person.png")
args = parser.parse_args()


# --------------------------
# Mediapipe face detection
# --------------------------
mp_face_detection = mp.solutions.face_detection

with mp_face_detection.FaceDetection(model_selection=0.5,
                                     min_detection_confidence=0.5) as face_detection:

    # -----------------------------------
    # MODE 1: REAL-TIME WEBCAM BLUR
    # -----------------------------------
    if args.mode == "webcam":
        cap = cv2.VideoCapture(0)

        if not cap.isOpened():
            print("❌ Webcam not found!")
            exit()

        print("🎥 Real-time face blurring started...")
        print("Press 'q' to quit.")

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            output = process_img(frame, face_detection)

            cv2.imshow("Face Blur (Webcam)", output)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()


🎥 Real-time face blurring started...
Press 'q' to quit.


In [1]:
import cv2; import mediapipe as mp; import numpy as np; print(cv2.__version__, mp.__version__, np.__version__)


4.8.1 0.10.21 1.26.4
